# Library

In [1]:
import random
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import itertools
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import joblib

### 건고추

In [2]:
train = pd.read_csv("train_건고추.csv", encoding = 'cp949')
test = pd.read_csv("test_건고추.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()


best_params_dict = {
    "1순": {
        9377: {"n_estimators": 1098, "max_depth": 11, "num_leaves": 101, "learning_rate": 0.014968766993417454, "min_child_samples": 21, "min_split_gain": 0.15208034013813845, "colsample_bytree": 0.7084304516095957, "subsample": 0.6002902178199586, "reg_alpha": 5.080809073863529, "reg_lambda": 3.3658807965961115},
        9555: {"n_estimators": 1194, "max_depth": 20, "num_leaves": 66, "learning_rate": 0.047536889692651685, "min_child_samples": 11, "min_split_gain": 0.6246855851352451, "colsample_bytree": 0.11528100820183972, "subsample": 0.6935905775004358, "reg_alpha": 4.497483360556183, "reg_lambda": 8.120056296244345},
        3949: {"n_estimators": 2225, "max_depth": 17, "num_leaves": 53, "learning_rate": 0.0962643205718953, "min_child_samples": 24, "min_split_gain": 0.4163722873686403, "colsample_bytree": 0.9747199886619341, "subsample": 0.8642700151119218, "reg_alpha": 6.324041249179467, "reg_lambda": 2.8234344906294124},
        7924: {"n_estimators": 1478, "max_depth": 14, "num_leaves": 89, "learning_rate": 0.012449904209621498, "min_child_samples": 24, "min_split_gain": 0.25207181775722987, "colsample_bytree": 0.2549997205325418, "subsample": 0.9576536181390767, "reg_alpha": 5.526099815070511, "reg_lambda": 8.272079215413315}
    },
    "2순": {
        9377: {"n_estimators": 918, "max_depth": 5, "num_leaves": 26, "learning_rate": 0.09570121888786312, "min_child_samples": 23, "min_split_gain": 0.8178260179617935, "colsample_bytree": 0.3394476168074535, "subsample": 0.7932684982397745, "reg_alpha": 0.04202608296611143, "reg_lambda": 1.882668234212162},
        9555: {"n_estimators": 2394, "max_depth": 5, "num_leaves": 114, "learning_rate": 0.20797697722665873, "min_child_samples": 25, "min_split_gain": 0.6391590790865345, "colsample_bytree": 0.30730304490883575, "subsample": 0.894310549278532, "reg_alpha": 2.0311866771977165, "reg_lambda": 3.395940549091655},
        3949: {"n_estimators": 1820, "max_depth": 20, "num_leaves": 96, "learning_rate": 0.20516714572591402, "min_child_samples": 7, "min_split_gain": 0.9673165047960778, "colsample_bytree": 0.15678635762508555, "subsample": 0.7608080347021827, "reg_alpha": 3.4056549087036982, "reg_lambda": 9.977534999503714},
        7924: {"n_estimators": 2222, "max_depth": 12, "num_leaves": 30, "learning_rate": 0.07564654150088453, "min_child_samples": 25, "min_split_gain": 0.07880626202781818, "colsample_bytree": 0.46774228798631506, "subsample": 0.9603569209885957, "reg_alpha": 0.9739864328587211, "reg_lambda": 2.1766780001898773}
    },
    "3순": {
        9377: {"n_estimators": 2564, "max_depth": 9, "num_leaves": 116, "learning_rate": 0.051367916517462114, "min_child_samples": 10, "min_split_gain": 0.5366410490073219, "colsample_bytree": 0.17891904103486397, "subsample": 0.9862198141947051, "reg_alpha": 1.9143068389441948, "reg_lambda": 9.697416290941831},
        9555: {"n_estimators": 674, "max_depth": 8, "num_leaves": 35, "learning_rate": 0.18631422133720993, "min_child_samples": 22, "min_split_gain": 0.17457377995239612, "colsample_bytree": 0.23348112178264552, "subsample": 0.5516427527971722, "reg_alpha": 3.5776302508053073, "reg_lambda": 5.10047630917752},
        3949: {"n_estimators": 646, "max_depth": 6, "num_leaves": 94, "learning_rate": 0.10407963619022532, "min_child_samples": 13, "min_split_gain": 0.47536750678020645, "colsample_bytree": 0.39982908486462837, "subsample": 0.8811857364812523, "reg_alpha": 0.28330124288045466, "reg_lambda": 9.699602398721263},
        7924: {"n_estimators": 734, "max_depth": 5, "num_leaves": 27, "learning_rate": 0.03770070107771281, "min_child_samples": 23, "min_split_gain": 0.5748098422650605, "colsample_bytree": 0.36299868821631065, "subsample": 0.8227778838486478, "reg_alpha": 6.443781975303593, "reg_lambda": 7.801184636023766}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'건고추_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  
sub_test = pd.read_csv('Processte.csv',encoding='cp949')
sub_test.loc[sub_test['품목명'] == '건고추', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '건고추', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '건고추', '3순'] = y_preds['y_pred_3']


### 감자

In [3]:
train = pd.read_csv("train_감자.csv", encoding = 'cp949')
test = pd.read_csv("test_감자.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2028, "max_depth": 9, "num_leaves": 126, "learning_rate": 0.11913547871651578, "min_child_samples": 14, "min_split_gain": 0.6258633589381968, "colsample_bytree": 0.646482040386875, "subsample": 0.5017800094085226, "reg_alpha": 5.976650168841801, "reg_lambda": 3.2550314927922344},
        9555: {"n_estimators": 909, "max_depth": 11, "num_leaves": 125, "learning_rate": 0.080671204703161, "min_child_samples": 5, "min_split_gain": 0.2081951090927474, "colsample_bytree": 0.16257669380992118, "subsample": 0.6532412518267251, "reg_alpha": 4.5475812341666675, "reg_lambda": 2.1501972765028476},
        3949: {"n_estimators": 2997, "max_depth": 18, "num_leaves": 42, "learning_rate": 0.25549539095281504, "min_child_samples": 27, "min_split_gain": 0.6215368906320398, "colsample_bytree": 0.25210528511849906, "subsample": 0.9271930642531467, "reg_alpha": 7.420462364700544, "reg_lambda": 7.7715736193158085},
        7924: {"n_estimators": 1067, "max_depth": 13, "num_leaves": 57, "learning_rate": 0.1999334205375976, "min_child_samples": 14, "min_split_gain": 0.4596000560118133, "colsample_bytree": 0.4852071562827866, "subsample": 0.8027943621616584, "reg_alpha": 5.785711381192529, "reg_lambda": 7.261630775914716}
    },
    "2순": {
        9377: {"n_estimators": 360, "max_depth": 11, "num_leaves": 74, "learning_rate": 0.07174119706582378, "min_child_samples": 10, "min_split_gain": 0.29781098857880917, "colsample_bytree": 0.5490548734629186, "subsample": 0.9375087118063085, "reg_alpha": 3.570397641752429, "reg_lambda": 1.5704717851120993},
        9555: {"n_estimators": 2325, "max_depth": 5, "num_leaves": 56, "learning_rate": 0.16272700690161557, "min_child_samples": 25, "min_split_gain": 0.5967790487239656, "colsample_bytree": 0.976034090416177, "subsample": 0.8004729190394896, "reg_alpha": 6.773800962591876, "reg_lambda": 9.095495468440081},
        3949: {"n_estimators": 1154, "max_depth": 20, "num_leaves": 96, "learning_rate": 0.21284893848346514, "min_child_samples": 25, "min_split_gain": 0.3690558011783258, "colsample_bytree": 0.6041709252848714, "subsample": 0.644834921976413, "reg_alpha": 4.762206971513262, "reg_lambda": 3.997576581619453},
        7924: {"n_estimators": 706, "max_depth": 10, "num_leaves": 90, "learning_rate": 0.191404591817877, "min_child_samples": 27, "min_split_gain": 0.9239666295354516, "colsample_bytree": 0.579366509011376, "subsample": 0.9301785619075332, "reg_alpha": 8.0695481592859, "reg_lambda": 1.180806269795621}
    },
    "3순": {
        9377: {"n_estimators": 626, "max_depth": 5, "num_leaves": 47, "learning_rate": 0.07792640810921478, "min_child_samples": 63, "min_split_gain": 0.7264481904443912, "colsample_bytree": 0.6096572910352505, "subsample": 0.7678728764501997, "reg_alpha": 1.527134709173917, "reg_lambda": 5.483893121129306},
        9555: {"n_estimators": 728, "max_depth": 4, "num_leaves": 113, "learning_rate": 0.053545869684277464, "min_child_samples": 65, "min_split_gain": 0.45425385734325996, "colsample_bytree": 0.23936545952419475, "subsample": 0.8306061607702225, "reg_alpha": 3.0920553896350547, "reg_lambda": 1.435566892246038},
        3949: {"n_estimators": 631, "max_depth": 6, "num_leaves": 44, "learning_rate": 0.20436583016875293, "min_child_samples": 64, "min_split_gain": 0.7314372603636262, "colsample_bytree": 0.624591560108012, "subsample": 0.8495014018123996, "reg_alpha": 6.3824693775393975, "reg_lambda": 3.976042010799203},
        7924: {"n_estimators": 533, "max_depth": 12, "num_leaves": 20, "learning_rate": 0.01292390115353695, "min_child_samples": 44, "min_split_gain": 0.23549368102402501, "colsample_bytree": 0.43428356402558266, "subsample": 0.8006390953473355, "reg_alpha": 7.75112997663998, "reg_lambda": 3.1931389991150465}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'감자_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  
    
sub_test.loc[sub_test['품목명'] == '감자', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '감자', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '감자', '3순'] = y_preds['y_pred_3']


### 깐마늘(국산)

In [4]:
train = pd.read_csv("train_깐마늘_국산.csv", encoding = 'cp949')
test = pd.read_csv("test_깐마늘_국산.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()


best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2804, "max_depth": 8, "num_leaves": 147, "learning_rate": 0.2859296555099065, "min_child_samples": 7, "min_split_gain": 0.45287644730330723, "colsample_bytree": 0.39106902111458464, "subsample": 0.5133257723495511, "reg_alpha": 9.787919612074749, "reg_lambda": 8.594445817613728},
        9555: {"n_estimators": 436, "max_depth": 20, "num_leaves": 56, "learning_rate": 0.18863472274620588, "min_child_samples": 7, "min_split_gain": 0.2447752314168098, "colsample_bytree": 0.73898721915043, "subsample": 0.9769630094807552, "reg_alpha": 8.407208810161723, "reg_lambda": 6.295860067702499},
        3949: {"n_estimators": 395, "max_depth": 20, "num_leaves": 20, "learning_rate": 0.2854197516461437, "min_child_samples": 7, "min_split_gain": 0.045818846038343436, "colsample_bytree": 0.4215879220630508, "subsample": 0.5043603609859049, "reg_alpha": 6.167291457457046, "reg_lambda": 7.354576782449882},
        7924: {"n_estimators": 2300, "max_depth": 9, "num_leaves": 110, "learning_rate": 0.19265622841561542, "min_child_samples": 7, "min_split_gain": 0.49117514201949586, "colsample_bytree": 0.7818510894795565, "subsample": 0.7909551741169032, "reg_alpha": 0.038765865606425365, "reg_lambda": 7.432127495874102}
    },
    "2순": {
        9377: {"n_estimators": 1272, "max_depth": 12, "num_leaves": 55, "learning_rate": 0.21127224041556253, "min_child_samples": 8, "min_split_gain": 0.27069168251552134, "colsample_bytree": 0.32424956425316825, "subsample": 0.5235378764512896, "reg_alpha": 6.495758815348248, "reg_lambda": 2.060917929033488},
        9555: {"n_estimators": 2960, "max_depth": 14, "num_leaves": 28, "learning_rate": 0.29702221376476545, "min_child_samples": 9, "min_split_gain": 0.9938962868171214, "colsample_bytree": 0.3866655109111251, "subsample": 0.5022335759286006, "reg_alpha": 1.8051815402965923, "reg_lambda": 0.14738583349219975},
        3949: {"n_estimators": 2376, "max_depth": 8, "num_leaves": 77, "learning_rate": 0.2695911723954397, "min_child_samples": 9, "min_split_gain": 0.21472010955338244, "colsample_bytree": 0.6342124411545131, "subsample": 0.6402282725315022, "reg_alpha": 4.381473735278596, "reg_lambda": 1.823102433459332},
        7924: {"n_estimators": 2903, "max_depth": 20, "num_leaves": 27, "learning_rate": 0.299663367771565, "min_child_samples": 5, "min_split_gain": 0.6991684176629079, "colsample_bytree": 0.1014761578678087, "subsample": 0.983392397909331, "reg_alpha": 0.14030512709632037, "reg_lambda": 0.06253559893368763}
    },
    "3순": {
        9377: {"n_estimators": 2885, "max_depth": 14, "num_leaves": 42, "learning_rate": 0.24093025966979584, "min_child_samples": 10, "min_split_gain": 0.15306097008792494, "colsample_bytree": 0.9024986417744453, "subsample": 0.9059882172726025, "reg_alpha": 5.595315371557593, "reg_lambda": 4.594342948947157},
        9555: {"n_estimators": 2324, "max_depth": 3, "num_leaves": 136, "learning_rate": 0.23921731541461932, "min_child_samples": 10, "min_split_gain": 0.44249839379023015, "colsample_bytree": 0.4042510236840241, "subsample": 0.8006552088799616, "reg_alpha": 6.235279888176862, "reg_lambda": 4.3928316828906},
        3949: {"n_estimators": 966, "max_depth": 8, "num_leaves": 21, "learning_rate": 0.164221209882672, "min_child_samples": 10, "min_split_gain": 0.2689996938840543, "colsample_bytree": 0.8245397165547479, "subsample": 0.9735022882852589, "reg_alpha": 4.512657503630595, "reg_lambda": 1.7313234250351222},
        7924: {"n_estimators": 1569, "max_depth": 13, "num_leaves": 75, "learning_rate": 0.1851084542642092, "min_child_samples": 10, "min_split_gain": 0.6962060654796146, "colsample_bytree": 0.7216876044989323, "subsample": 0.836753805586268, "reg_alpha": 7.636489939719693, "reg_lambda": 6.471075341567903}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'깐마늘(국산)_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred 

sub_test.loc[sub_test['품목명'] == '깐마늘(국산)', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '깐마늘(국산)', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '깐마늘(국산)', '3순'] = y_preds['y_pred_3']


### 대파

In [5]:
train = pd.read_csv("train_대파.csv", encoding = 'cp949')
test = pd.read_csv("test_대파.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()


best_params_dict= {
    "1순": {
        9377: {"n_estimators": 1726, "max_depth": 10, "num_leaves": 100, "learning_rate": 0.2708589244239474, "min_child_samples": 73, "min_split_gain": 0.7863031822837507, "colsample_bytree": 0.454307092302608, "subsample": 0.7026951549512084, "reg_alpha": 2.6779280824906007, "reg_lambda": 2.1685330301148573},
        9555: {"n_estimators": 1468, "max_depth": 11, "num_leaves": 40, "learning_rate": 0.2339278784702823, "min_child_samples": 70, "min_split_gain": 0.21023861938792776, "colsample_bytree": 0.5128734782824731, "subsample": 0.7231063622760838, "reg_alpha": 3.7361601177677817, "reg_lambda": 1.5150671069836719},
        3949: {"n_estimators": 553, "max_depth": 3, "num_leaves": 51, "learning_rate": 0.07556820071534895, "min_child_samples": 5, "min_split_gain": 0.2563544664393151, "colsample_bytree": 0.8440988927621849, "subsample": 0.9979613055802015, "reg_alpha": 8.993260757697294, "reg_lambda": 7.935571173613591},
        7924: {"n_estimators": 405, "max_depth": 17, "num_leaves": 117, "learning_rate": 0.08921858303941999, "min_child_samples": 95, "min_split_gain": 0.4544718211169432, "colsample_bytree": 0.46713952423286775, "subsample": 0.8936815793237936, "reg_alpha": 6.538179148658433, "reg_lambda": 0.11619852951381904}
    },
    "2순": {
        9377: {"n_estimators": 1920, "max_depth": 15, "num_leaves": 91, "learning_rate": 0.09471793391547356, "min_child_samples": 90, "min_split_gain": 0.38195766315787316, "colsample_bytree": 0.8156788747304593, "subsample": 0.887450423910191, "reg_alpha": 6.766518282641253, "reg_lambda": 8.293163356686444},
        9555: {"n_estimators": 529, "max_depth": 20, "num_leaves": 63, "learning_rate": 0.2205498854767516, "min_child_samples": 82, "min_split_gain": 0.7730128923094379, "colsample_bytree": 0.8315593002838223, "subsample": 0.6463975550151886, "reg_alpha": 2.3709401019029697, "reg_lambda": 2.3166011134241273},
        3949: {"n_estimators": 351, "max_depth": 7, "num_leaves": 101, "learning_rate": 0.26265920403646553, "min_child_samples": 85, "min_split_gain": 0.8308429652741786, "colsample_bytree": 0.36728849625826754, "subsample": 0.85259492141903, "reg_alpha": 6.742018064541768, "reg_lambda": 3.788221065749533},
        7924: {"n_estimators": 2995, "max_depth": 5, "num_leaves": 72, "learning_rate": 0.20162185915498074, "min_child_samples": 72, "min_split_gain": 0.22040719266594744, "colsample_bytree": 0.5457775683381185, "subsample": 0.6368106731034686, "reg_alpha": 7.288111202832229, "reg_lambda": 0.640529415493325}
    },
    "3순": {
        9377: {"n_estimators": 876, "max_depth": 8, "num_leaves": 102, "learning_rate": 0.06665449353386822, "min_child_samples": 92, "min_split_gain": 0.48731883651707897, "colsample_bytree": 0.3775404750561995, "subsample": 0.6945386656510149, "reg_alpha": 9.32383874726868, "reg_lambda": 0.7642688169203382},
        9555: {"n_estimators": 1490, "max_depth": 6, "num_leaves": 125, "learning_rate": 0.07140592985986131, "min_child_samples": 91, "min_split_gain": 0.6934972582017358, "colsample_bytree": 0.9592889806973043, "subsample": 0.876891226438197, "reg_alpha": 2.152989139128546, "reg_lambda": 1.332207451402384},
        3949: {"n_estimators": 2860, "max_depth": 19, "num_leaves": 62, "learning_rate": 0.2426426293784521, "min_child_samples": 89, "min_split_gain": 0.6564316001092004, "colsample_bytree": 0.7305718672510867, "subsample": 0.9732578702364912, "reg_alpha": 8.151453525759877, "reg_lambda": 5.7363698937002745},
        7924: {"n_estimators": 935, "max_depth": 15, "num_leaves": 68, "learning_rate": 0.1298986285166347, "min_child_samples": 91, "min_split_gain": 0.8922485931196327, "colsample_bytree": 0.9714253624153929, "subsample": 0.5918376396098393, "reg_alpha": 0.8238061412623887, "reg_lambda": 9.036550894274516}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'대파_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '대파', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '대파', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '대파', '3순'] = y_preds['y_pred_3']


### 무

In [6]:
train = pd.read_csv("train_무.csv", encoding = 'cp949')
test = pd.read_csv("test_무.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2692, "max_depth": 8, "num_leaves": 133, "learning_rate": 0.2294854541898778, "min_child_samples": 12, "min_split_gain": 0.8222510808723222, "colsample_bytree": 0.8458056047944902, "subsample": 0.653969921401943, "reg_alpha": 5.350706996448639, "reg_lambda": 0.1687903062175475},
        9555: {"n_estimators": 2398, "max_depth": 18, "num_leaves": 46, "learning_rate": 0.2859125249645124, "min_child_samples": 16, "min_split_gain": 0.41686110540989424, "colsample_bytree": 0.8003485679123377, "subsample": 0.7151864947552022, "reg_alpha": 8.090177453237366, "reg_lambda": 9.43914208423287},
        3949: {"n_estimators": 471, "max_depth": 4, "num_leaves": 107, "learning_rate": 0.1587868719217326, "min_child_samples": 15, "min_split_gain": 0.46715191698958736, "colsample_bytree": 0.6208562113779635, "subsample": 0.9090951237058353, "reg_alpha": 8.60302037928253, "reg_lambda": 8.793690563785846},
        7924: {"n_estimators": 1740, "max_depth": 10, "num_leaves": 97, "learning_rate": 0.08577290294705212, "min_child_samples": 17, "min_split_gain": 0.25090000133420737, "colsample_bytree": 0.6945464480286426, "subsample": 0.951110605670805, "reg_alpha": 9.785663015451469, "reg_lambda": 1.442200406959161}
    },
    "2순": {
        9377: {"n_estimators": 1820, "max_depth": 6, "num_leaves": 68, "learning_rate": 0.15732075456721947, "min_child_samples": 13, "min_split_gain": 0.7283018087066382, "colsample_bytree": 0.218414408460938, "subsample": 0.9048044882788268, "reg_alpha": 7.1117998328333245, "reg_lambda": 0.0019139600114803645},
        9555: {"n_estimators": 1821, "max_depth": 16, "num_leaves": 89, "learning_rate": 0.060839467803958006, "min_child_samples": 16, "min_split_gain": 0.8744946293147635, "colsample_bytree": 0.49247962565580006, "subsample": 0.7982654096865838, "reg_alpha": 4.116273423647645, "reg_lambda": 6.721180702711253},
        3949: {"n_estimators": 790, "max_depth": 16, "num_leaves": 91, "learning_rate": 0.010671941866701338, "min_child_samples": 22, "min_split_gain": 0.7717883135518575, "colsample_bytree": 0.331339854545188, "subsample": 0.9117386475791461, "reg_alpha": 1.8492907462838948, "reg_lambda": 8.179505408434835},
        7924: {"n_estimators": 1179, "max_depth": 9, "num_leaves": 139, "learning_rate": 0.05286758241297887, "min_child_samples": 56, "min_split_gain": 0.12688002877540952, "colsample_bytree": 0.3314047391930844, "subsample": 0.91849624203601, "reg_alpha": 6.757936744936707, "reg_lambda": 8.94631089768132}
    },
    "3순": {
        9377: {"n_estimators": 2457, "max_depth": 18, "num_leaves": 105, "learning_rate": 0.18419964732283123, "min_child_samples": 5, "min_split_gain": 0.8395087069662488, "colsample_bytree": 0.23702549479862706, "subsample": 0.87831154211986, "reg_alpha": 9.931137197527484, "reg_lambda": 6.952786013979078},
        9555: {"n_estimators": 1169, "max_depth": 16, "num_leaves": 104, "learning_rate": 0.06993863316921935, "min_child_samples": 66, "min_split_gain": 0.9632231820861599, "colsample_bytree": 0.8639360385947754, "subsample": 0.6681241607418659, "reg_alpha": 3.409588893483952, "reg_lambda": 1.8457220121360427},
        3949: {"n_estimators": 2150, "max_depth": 20, "num_leaves": 105, "learning_rate": 0.01091004766009974, "min_child_samples": 57, "min_split_gain": 0.9140990115952892, "colsample_bytree": 0.4582696983074369, "subsample": 0.8003907937844236, "reg_alpha": 4.16359086991916, "reg_lambda": 1.7611576878151194},
        7924: {"n_estimators": 711, "max_depth": 16, "num_leaves": 22, "learning_rate": 0.03169931175895504, "min_child_samples": 63, "min_split_gain": 0.2784198629408612, "colsample_bytree": 0.8871585347974954, "subsample": 0.7693930559392297, "reg_alpha": 8.72228612424313, "reg_lambda": 8.0701499360161}
    }
}


seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'무_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '무', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '무', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '무', '3순'] = y_preds['y_pred_3']


### 배추

In [7]:
train = pd.read_csv("train_배추.csv", encoding = 'cp949')
test = pd.read_csv("test_배추.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 322, "max_depth": 3, "num_leaves": 150, "learning_rate": 0.15847925482038785, "min_child_samples": 5, "min_split_gain": 0.350240437411248, "colsample_bytree": 0.37375319366607446, "subsample": 0.6772336531131276, "reg_alpha": 9.619153018173987, "reg_lambda": 0.029960737199986376},
        9555: {"n_estimators": 1487, "max_depth": 18, "num_leaves": 139, "learning_rate": 0.030430756656479235, "min_child_samples": 11, "min_split_gain": 0.06608667016902012, "colsample_bytree": 0.5279228445739284, "subsample": 0.918764514780781, "reg_alpha": 7.393888697665528, "reg_lambda": 7.661514339370825},
        3949: {"n_estimators": 2230, "max_depth": 15, "num_leaves": 34, "learning_rate": 0.15870410133861343, "min_child_samples": 12, "min_split_gain": 0.4335848126819511, "colsample_bytree": 0.5840274151619375, "subsample": 0.6870925512601304, "reg_alpha": 3.0360408556018283, "reg_lambda": 2.969259375012628},
        7924: {"n_estimators": 2822, "max_depth": 18, "num_leaves": 110, "learning_rate": 0.19104824013933494, "min_child_samples": 11, "min_split_gain": 0.17206455722613095, "colsample_bytree": 0.7472758334779057, "subsample": 0.811686675450502, "reg_alpha": 9.399902128206918, "reg_lambda": 7.978626202824703}
    },
    "2순": {
        9377: {"n_estimators": 2956, "max_depth": 16, "num_leaves": 21, "learning_rate": 0.23485030065523854, "min_child_samples": 6, "min_split_gain": 0.23205511062721823, "colsample_bytree": 0.9984495372419513, "subsample": 0.5030525666877903, "reg_alpha": 1.895413640919017, "reg_lambda": 3.984282211133565},
        9555: {"n_estimators": 2697, "max_depth": 9, "num_leaves": 144, "learning_rate": 0.07073302904412358, "min_child_samples": 26, "min_split_gain": 0.5720830958363409, "colsample_bytree": 0.5987962935882291, "subsample": 0.9188698802112736, "reg_alpha": 9.88876153424182, "reg_lambda": 6.020150934834949},
        3949: {"n_estimators": 1057, "max_depth": 3, "num_leaves": 48, "learning_rate": 0.25849140488379263, "min_child_samples": 6, "min_split_gain": 0.28745492647035337, "colsample_bytree": 0.5154500201827691, "subsample": 0.8750767410491704, "reg_alpha": 3.250967633430153, "reg_lambda": 5.479943286150642},
        7924: {"n_estimators": 2975, "max_depth": 11, "num_leaves": 42, "learning_rate": 0.24306994512868324, "min_child_samples": 7, "min_split_gain": 0.9912181981263855, "colsample_bytree": 0.8375397956226178, "subsample": 0.6883970333415558, "reg_alpha": 8.790017714299225, "reg_lambda": 9.844274964903006}
    },
    "3순": {
        9377: {"n_estimators": 2985, "max_depth": 16, "num_leaves": 69, "learning_rate": 0.02274999552065067, "min_child_samples": 9, "min_split_gain": 0.9989713642347254, "colsample_bytree": 0.13977753918649818, "subsample": 0.9402022364564175, "reg_alpha": 6.946173002251799, "reg_lambda": 3.5962494830652565},
        9555: {"n_estimators": 1848, "max_depth": 8, "num_leaves": 65, "learning_rate": 0.09058390271820418, "min_child_samples": 5, "min_split_gain": 0.8120930456343977, "colsample_bytree": 0.14580865215584998, "subsample": 0.9905511495098699, "reg_alpha": 7.457703159795409, "reg_lambda": 1.382877690531101},
        3949: {"n_estimators": 2302, "max_depth": 6, "num_leaves": 76, "learning_rate": 0.036434868681663804, "min_child_samples": 15, "min_split_gain": 0.7338006009848799, "colsample_bytree": 0.9010695268150053, "subsample": 0.7454855176143877, "reg_alpha": 6.26095134947194, "reg_lambda": 7.292308356066771},
        7924: {"n_estimators": 425, "max_depth": 4, "num_leaves": 119, "learning_rate": 0.011182850615129407, "min_child_samples": 13, "min_split_gain": 0.2650595630252986, "colsample_bytree": 0.9021911718002138, "subsample": 0.5024953907274472, "reg_alpha": 8.39261475897881, "reg_lambda": 8.674531690847004}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'배추_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '배추', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '배추', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '배추', '3순'] = y_preds['y_pred_3']


### 사과

In [8]:
train = pd.read_csv("train_사과.csv", encoding = 'cp949')
test = pd.read_csv("test_사과.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()


best_params_dict= {
    "1순": {
        9377: {"n_estimators": 1286, "max_depth": 14, "num_leaves": 118, "learning_rate": 0.1485140457441414, "min_child_samples": 24, "min_split_gain": 0.6551012412254746, "colsample_bytree": 0.8628751309020326, "subsample": 0.7665931586126593, "reg_alpha": 8.775517882210144, "reg_lambda": 4.850142868320802},
        9555: {"n_estimators": 2104, "max_depth": 4, "num_leaves": 96, "learning_rate": 0.050428355167126834, "min_child_samples": 24, "min_split_gain": 0.5377744366206951, "colsample_bytree": 0.8820415506902072, "subsample": 0.9038280440504677, "reg_alpha": 2.93561050734753, "reg_lambda": 7.834663314803874},
        3949: {"n_estimators": 1424, "max_depth": 15, "num_leaves": 83, "learning_rate": 0.03206181732639185, "min_child_samples": 66, "min_split_gain": 0.6744525862804438, "colsample_bytree": 0.5616552686591305, "subsample": 0.9915527699202108, "reg_alpha": 4.599977459015533, "reg_lambda": 5.709169424212267},
        7924: {"n_estimators": 343, "max_depth": 14, "num_leaves": 132, "learning_rate": 0.010254287597383671, "min_child_samples": 13, "min_split_gain": 0.9666030505152453, "colsample_bytree": 0.9095298620955681, "subsample": 0.5886370600629891, "reg_alpha": 0.9811435902213321, "reg_lambda": 5.6106743750335415}
    },
    "2순": {
        9377: {"n_estimators": 498, "max_depth": 12, "num_leaves": 72, "learning_rate": 0.1917595924406815, "min_child_samples": 64, "min_split_gain": 0.7604948609442812, "colsample_bytree": 0.4511719254062966, "subsample": 0.5633788583980868, "reg_alpha": 9.851933738392011, "reg_lambda": 0.6230185101077979},
        9555: {"n_estimators": 486, "max_depth": 11, "num_leaves": 44, "learning_rate": 0.18299183474547925, "min_child_samples": 64, "min_split_gain": 0.9023347719162182, "colsample_bytree": 0.2798301041055288, "subsample": 0.9102953927468181, "reg_alpha": 5.800030252009352, "reg_lambda": 6.496376187127129},
        3949: {"n_estimators": 2026, "max_depth": 7, "num_leaves": 35, "learning_rate": 0.11544462782996255, "min_child_samples": 64, "min_split_gain": 0.06354602469992106, "colsample_bytree": 0.6063456793575958, "subsample": 0.5899100068307054, "reg_alpha": 5.063268672807034, "reg_lambda": 5.952113248822677},
        7924: {"n_estimators": 2594, "max_depth": 14, "num_leaves": 117, "learning_rate": 0.028773885639794425, "min_child_samples": 61, "min_split_gain": 0.5876293689096052, "colsample_bytree": 0.10309737618691449, "subsample": 0.7825244820281811, "reg_alpha": 5.263616748087239, "reg_lambda": 3.6444867559303646}
    },
    "3순": {
        9377: {"n_estimators": 313, "max_depth": 20, "num_leaves": 52, "learning_rate": 0.11250383030010422, "min_child_samples": 62, "min_split_gain": 0.44086465814396114, "colsample_bytree": 0.13886171394531316, "subsample": 0.5988568654532425, "reg_alpha": 9.573502939647481, "reg_lambda": 3.031563766893375},
        9555: {"n_estimators": 1001, "max_depth": 16, "num_leaves": 140, "learning_rate": 0.14568491105829445, "min_child_samples": 59, "min_split_gain": 0.8787676153133409, "colsample_bytree": 0.15799732903817226, "subsample": 0.8965499008284693, "reg_alpha": 0.5074204894625158, "reg_lambda": 6.026296844875644},
        3949: {"n_estimators": 930, "max_depth": 15, "num_leaves": 49, "learning_rate": 0.01082044400021246, "min_child_samples": 54, "min_split_gain": 0.2570133346160276, "colsample_bytree": 0.8696053420600748, "subsample": 0.8844781839501901, "reg_alpha": 5.535264041496899, "reg_lambda": 7.006605906601179},
        7924: {"n_estimators": 443, "max_depth": 9, "num_leaves": 117, "learning_rate": 0.026430766873997043, "min_child_samples": 55, "min_split_gain": 0.08526350027751506, "colsample_bytree": 0.7657685722305556, "subsample": 0.9068515265126321, "reg_alpha": 9.385735974603808, "reg_lambda": 7.494237833789069}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'사과_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '사과', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '사과', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '사과', '3순'] = y_preds['y_pred_3']


### 상추

In [9]:
train = pd.read_csv("train_상추.csv", encoding = 'cp949')
test = pd.read_csv("test_상추.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()


best_params_dict= {
    "1순": {
        9377: {"n_estimators": 737, "max_depth": 13, "num_leaves": 141, "learning_rate": 0.20900500847155332, "min_child_samples": 34, "min_split_gain": 0.8655594150283508, "colsample_bytree": 0.991861778082882, "subsample": 0.9098031590576832, "reg_alpha": 9.589010028815808, "reg_lambda": 4.371589052460859},
        9555: {"n_estimators": 512, "max_depth": 7, "num_leaves": 114, "learning_rate": 0.1599784672398502, "min_child_samples": 33, "min_split_gain": 0.8657416365488965, "colsample_bytree": 0.5147159477335562, "subsample": 0.7763153733952561, "reg_alpha": 9.080926719210353, "reg_lambda": 4.439747410593142},
        3949: {"n_estimators": 1505, "max_depth": 20, "num_leaves": 31, "learning_rate": 0.0234717660592568, "min_child_samples": 32, "min_split_gain": 0.07907396445558118, "colsample_bytree": 0.7096952197331418, "subsample": 0.8198978894195935, "reg_alpha": 4.17958187527251, "reg_lambda": 7.78821841303745},
        7924: {"n_estimators": 1301, "max_depth": 15, "num_leaves": 44, "learning_rate": 0.04464465036086504, "min_child_samples": 34, "min_split_gain": 0.6966114131086617, "colsample_bytree": 0.6303021504862703, "subsample": 0.9969615296656009, "reg_alpha": 4.1579609668543975, "reg_lambda": 6.664817786683813}
    },
    "2순": {
        9377: {"n_estimators": 1922, "max_depth": 12, "num_leaves": 74, "learning_rate": 0.056608562814697505, "min_child_samples": 38, "min_split_gain": 0.5255811905954905, "colsample_bytree": 0.32430578037234614, "subsample": 0.9317106065252928, "reg_alpha": 2.3904316171556186, "reg_lambda": 7.395527333071904},
        9555: {"n_estimators": 578, "max_depth": 18, "num_leaves": 123, "learning_rate": 0.1306242594285764, "min_child_samples": 51, "min_split_gain": 0.9616550190287256, "colsample_bytree": 0.6618676464372667, "subsample": 0.7659806812115669, "reg_alpha": 2.659121134975158, "reg_lambda": 1.2260283945012778},
        3949: {"n_estimators": 976, "max_depth": 17, "num_leaves": 113, "learning_rate": 0.05547435989055123, "min_child_samples": 38, "min_split_gain": 0.5263447745813508, "colsample_bytree": 0.3701501009528873, "subsample": 0.5175082223185927, "reg_alpha": 0.05585493174367295, "reg_lambda": 9.484861950875938},
        7924: {"n_estimators": 1361, "max_depth": 11, "num_leaves": 77, "learning_rate": 0.12627971839426055, "min_child_samples": 24, "min_split_gain": 0.9318114324358681, "colsample_bytree": 0.35514396832318107, "subsample": 0.6360223901357013, "reg_alpha": 0.011855231613867478, "reg_lambda": 1.59629826485667}
    },
    "3순": {
        9377: {"n_estimators": 2045, "max_depth": 13, "num_leaves": 105, "learning_rate": 0.1343105227896428, "min_child_samples": 32, "min_split_gain": 0.23383566373635553, "colsample_bytree": 0.7099757231977502, "subsample": 0.8751666566619584, "reg_alpha": 5.4318649402714865, "reg_lambda": 4.873058641257871},
        9555: {"n_estimators": 2076, "max_depth": 11, "num_leaves": 74, "learning_rate": 0.09052994235903256, "min_child_samples": 25, "min_split_gain": 0.06155921665708691, "colsample_bytree": 0.2211552713857181, "subsample": 0.832623890341881, "reg_alpha": 6.703877334356463, "reg_lambda": 4.871423176243574},
        3949: {"n_estimators": 1676, "max_depth": 10, "num_leaves": 40, "learning_rate": 0.18111927325190633, "min_child_samples": 24, "min_split_gain": 0.7914618062228923, "colsample_bytree": 0.8260301551608816, "subsample": 0.836446448102171, "reg_alpha": 1.2396719269631822, "reg_lambda": 6.550204757738436},
        7924: {"n_estimators": 728, "max_depth": 11, "num_leaves": 134, "learning_rate": 0.12358056695551871, "min_child_samples": 27, "min_split_gain": 0.8362577783212037, "colsample_bytree": 0.442211999573338, "subsample": 0.6654275375668897, "reg_alpha": 7.869255263857983, "reg_lambda": 4.335416378292042}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4): 
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'상추_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '상추', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '상추', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '상추', '3순'] = y_preds['y_pred_3']


### 양파

In [10]:
train = pd.read_csv("train_양파.csv", encoding = 'cp949')
test = pd.read_csv("test_양파.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 1910, "max_depth": 17, "num_leaves": 27, "learning_rate": 0.2778944087176605, "min_child_samples": 23, "min_split_gain": 0.8366833634781942, "colsample_bytree": 0.11035012821000795, "subsample": 0.8654798844587429, "reg_alpha": 8.742365539241412, "reg_lambda": 9.540687835922265},
        9555: {"n_estimators": 903, "max_depth": 8, "num_leaves": 148, "learning_rate": 0.014043455773646352, "min_child_samples": 24, "min_split_gain": 0.9878085421526658, "colsample_bytree": 0.671634075234645, "subsample": 0.6655213803264318, "reg_alpha": 7.449383573704951, "reg_lambda": 7.3492930166759},
        3949: {"n_estimators": 1917, "max_depth": 18, "num_leaves": 36, "learning_rate": 0.12680464060937174, "min_child_samples": 24, "min_split_gain": 0.6267322400401307, "colsample_bytree": 0.218787107639265, "subsample": 0.5993934476480122, "reg_alpha": 7.357493195327498, "reg_lambda": 5.574439729031217},
        7924: {"n_estimators": 1838, "max_depth": 14, "num_leaves": 139, "learning_rate": 0.13083263435020268, "min_child_samples": 23, "min_split_gain": 0.5189241756798517, "colsample_bytree": 0.5188205960885365, "subsample": 0.6384878106593145, "reg_alpha": 8.475282715170453, "reg_lambda": 7.877193547981843}
    },
    "2순": {
        9377: {"n_estimators": 2107, "max_depth": 17, "num_leaves": 124, "learning_rate": 0.27611571042812777, "min_child_samples": 23, "min_split_gain": 0.8324915938614418, "colsample_bytree": 0.22920068223721918, "subsample": 0.8587152965519016, "reg_alpha": 4.837329870425256, "reg_lambda": 6.764787113420978},
        9555: {"n_estimators": 2055, "max_depth": 6, "num_leaves": 129, "learning_rate": 0.12602237378532666, "min_child_samples": 22, "min_split_gain": 0.6151863888287236, "colsample_bytree": 0.1195071240226382, "subsample": 0.6289805741515723, "reg_alpha": 7.432494165083835, "reg_lambda": 0.8304839630302345},
        3949: {"n_estimators": 1954, "max_depth": 3, "num_leaves": 97, "learning_rate": 0.2996799686568136, "min_child_samples": 25, "min_split_gain": 0.5402911240368107, "colsample_bytree": 0.21244875241267264, "subsample": 0.7257739195016127, "reg_alpha": 3.202397848034875, "reg_lambda": 2.2181193133678283},
        7924: {"n_estimators": 2476, "max_depth": 20, "num_leaves": 142, "learning_rate": 0.19896668374102797, "min_child_samples": 24, "min_split_gain": 0.8013588631035355, "colsample_bytree": 0.12122678993328131, "subsample": 0.9377504731525347, "reg_alpha": 5.9332430425094955, "reg_lambda": 9.207412020646606}
    },
    "3순": {
        9377: {"n_estimators": 1847, "max_depth": 7, "num_leaves": 38, "learning_rate": 0.23912060442531824, "min_child_samples": 27, "min_split_gain": 0.3574812139254765, "colsample_bytree": 0.26608945958105396, "subsample": 0.6303967246703093, "reg_alpha": 3.4108964869951515, "reg_lambda": 5.77633480727249},
        9555: {"n_estimators": 2145, "max_depth": 20, "num_leaves": 118, "learning_rate": 0.07208738086799424, "min_child_samples": 24, "min_split_gain": 0.3854555145974067, "colsample_bytree": 0.3692071761835134, "subsample": 0.69098012038176, "reg_alpha": 7.7333713920916, "reg_lambda": 0.15475789005028506},
        3949: {"n_estimators": 855, "max_depth": 8, "num_leaves": 24, "learning_rate": 0.25243686161929346, "min_child_samples": 11, "min_split_gain": 0.2075582029511296, "colsample_bytree": 0.3451100332953668, "subsample": 0.8628215502671137, "reg_alpha": 2.8486783986532296, "reg_lambda": 3.529120432450821},
        7924: {"n_estimators": 2720, "max_depth": 3, "num_leaves": 35, "learning_rate": 0.012233501137072805, "min_child_samples": 25, "min_split_gain": 0.24751582512635806, "colsample_bytree": 0.998550031649408, "subsample": 0.8974303060979885, "reg_alpha": 4.167086039394305, "reg_lambda": 3.058995172871472}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순' 

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'양파_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred 

sub_test.loc[sub_test['품목명'] == '양파', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '양파', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '양파', '3순'] = y_preds['y_pred_3']


### 배

In [11]:
train = pd.read_csv("train_배.csv", encoding = 'cp949')
test = pd.read_csv("test_배.csv", encoding = 'cp949')

train.drop(['시점'],axis=1,inplace=True)
test.drop(['시점'],axis=1,inplace=True)
test.drop(['1순','2순','3순'],axis=1,inplace=True)

columns_to_drop = train.select_dtypes(include=['object','category']).columns.tolist()
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
total_samples = len(train)
features_to_drop = [col for col in train.columns if (train[col] == 0).sum() / total_samples >= 0.3]
train = train.drop(columns=features_to_drop)
test = test.drop(columns=features_to_drop)

y_preds = {}
train_x = train.drop(['1순', '2순', '3순'], axis=1)
train_y = train[['1순', '2순', '3순']].copy()

best_params_dict= {
    "1순": {
        9377: {"n_estimators": 2959, "max_depth": 5, "num_leaves": 93, "learning_rate": 0.1807086742241495, "min_child_samples": 50, "min_split_gain": 0.4587701289726267, "colsample_bytree": 0.8878681882766268, "subsample": 0.8380134648692271, "reg_alpha": 4.716374580966418, "reg_lambda": 4.0887877311576695},
        9555: {"n_estimators": 932, "max_depth": 4, "num_leaves": 32, "learning_rate": 0.267400718829073, "min_child_samples": 42, "min_split_gain": 0.530995199434697, "colsample_bytree": 0.1902696003354979, "subsample": 0.9373514343406253, "reg_alpha": 8.277538806217411, "reg_lambda": 4.229909141295841},
        3949: {"n_estimators": 970, "max_depth": 4, "num_leaves": 111, "learning_rate": 0.1072228105871125, "min_child_samples": 40, "min_split_gain": 0.11856075289097134, "colsample_bytree": 0.17488071527167853, "subsample": 0.670169996583688, "reg_alpha": 6.0874204162326215, "reg_lambda": 5.577346061852932},
        7924: {"n_estimators": 2997, "max_depth": 11, "num_leaves": 109, "learning_rate": 0.15171067138326094, "min_child_samples": 53, "min_split_gain": 0.6664240596253966, "colsample_bytree": 0.23568937063759643, "subsample": 0.7548427486931037, "reg_alpha": 7.220461477620253, "reg_lambda": 0.6741436299176772}
    },
    "2순": {
        9377: {"n_estimators": 2370, "max_depth": 16, "num_leaves": 90, "learning_rate": 0.0521408516370903, "min_child_samples": 44, "min_split_gain": 0.5450794237491235, "colsample_bytree": 0.9083406368435625, "subsample": 0.6058454386146588, "reg_alpha": 0.004320691399196885, "reg_lambda": 2.9813347844414335},
        9555: {"n_estimators": 398, "max_depth": 14, "num_leaves": 113, "learning_rate": 0.21790205486742315, "min_child_samples": 43, "min_split_gain": 0.215178062367526, "colsample_bytree": 0.3596847617009553, "subsample": 0.6456384937970279, "reg_alpha": 1.674461213562497, "reg_lambda": 5.246428923845883},
        3949: {"n_estimators": 763, "max_depth": 5, "num_leaves": 48, "learning_rate": 0.25864956691704166, "min_child_samples": 44, "min_split_gain": 0.6717223942118652, "colsample_bytree": 0.5039957614067565, "subsample": 0.5596628966173678, "reg_alpha": 7.722146312071906, "reg_lambda": 5.5065613423163615},
        7924: {"n_estimators": 1766, "max_depth": 18, "num_leaves": 62, "learning_rate": 0.07351911859361084, "min_child_samples": 44, "min_split_gain": 0.4266011588764111, "colsample_bytree": 0.6013127182560822, "subsample": 0.950236186181326, "reg_alpha": 1.1894390063602454, "reg_lambda": 3.3133037742745386}
    },
    "3순": {
        9377: {"n_estimators": 307, "max_depth": 13, "num_leaves": 124, "learning_rate": 0.23492131068477365, "min_child_samples": 47, "min_split_gain": 0.08414155219675867, "colsample_bytree": 0.566919979647655, "subsample": 0.848923850165228, "reg_alpha": 1.7170811437698912, "reg_lambda": 7.829390330870659},
        9555: {"n_estimators": 576, "max_depth": 6, "num_leaves": 109, "learning_rate": 0.09938099656938074, "min_child_samples": 47, "min_split_gain": 0.33492194259332436, "colsample_bytree": 0.6695289134672294, "subsample": 0.7379310469461808, "reg_alpha": 8.592146169561527, "reg_lambda": 3.006032091803984},
        3949: {"n_estimators": 2567, "max_depth": 3, "num_leaves": 73, "learning_rate": 0.1483421807061788, "min_child_samples": 11, "min_split_gain": 0.7540882994825181, "colsample_bytree": 0.9976915459809096, "subsample": 0.7734909304836903, "reg_alpha": 8.266607147191316, "reg_lambda": 0.8863367975890868},
        7924: {"n_estimators": 1279, "max_depth": 17, "num_leaves": 138, "learning_rate": 0.18756145080482126, "min_child_samples": 47, "min_split_gain": 0.8120233044433158, "colsample_bytree": 0.6778775175185686, "subsample": 0.7744501827311342, "reg_alpha": 5.950373499846679, "reg_lambda": 3.5688599201052233}
    }
}

seeds = [9377, 9555, 3949, 7924]

for i in range(1, 4):  
    seed_test_preds = []
    순_key = f'{i}순'  

    for seed in seeds:
        train_set = pd.concat([train_x, train_y[순_key]], axis=1)
        train_set = train_set[train_set[순_key] > 0]

        X = train_set.drop(columns=[순_key])
        y = train_set[순_key]

        best_params = best_params_dict[순_key][seed]
        best_params['random_state'] = seed 
        best_model = lgb.LGBMRegressor(**best_params)
        best_model.fit(X, y)

        model_save_path = f'배_LGBM_model_{i}순_{seed}.pkl'
        joblib.dump(best_model, model_save_path)
        seed_test_preds.append(best_model.predict(test))

    final_test_pred = np.mean(seed_test_preds, axis=0)
    y_preds[f'y_pred_{i}'] = final_test_pred  

sub_test.loc[sub_test['품목명'] == '배', '1순'] = y_preds['y_pred_1']
sub_test.loc[sub_test['품목명'] == '배', '2순'] = y_preds['y_pred_2']
sub_test.loc[sub_test['품목명'] == '배', '3순'] = y_preds['y_pred_3']


# Final Sub

In [12]:
submission_path = 'sample_submission.csv'
final_test_data = sub_test.copy()
submission_data = pd.read_csv(submission_path, encoding='utf-8')
test_columns = ['시점', '품목명', '1순', '2순', '3순']
final_test_filtered = final_test_data[test_columns]
pivot_test_data = final_test_filtered.pivot(index='시점', columns='품목명', values=['1순', '2순', '3순'])
for step in ['1순', '2순', '3순']:
    for product in submission_data.columns[1:]:  
        col_name = f'{step}'
        submission_data.loc[submission_data['시점'].str.contains(step, regex=False), product] = pivot_test_data[col_name, product].values
submission_data.to_csv('LGBM.csv', index=False, encoding='utf-8')
submission_data

,시점,감자,건고추,깐마늘(국산),대파,무,배추,사과,상추,양파,배
0,TEST_00+1순,36867.356945,706891.663882,169086.279218,1995.798101,20026.950073,13358.439738,24332.272216,1186.114320,1437.042060,29469.487096
1,TEST_00+2순,38533.196504,690569.874812,169157.955693,1807.629747,16164.447534,7805.255206,21914.116453,1292.319558,1326.823913,29230.683119
2,TEST_00+3순,39187.287533,662452.368637,168874.425219,1811.650465,12404.514293,6846.623471,22421.087663,1116.195565,1232.890091,30895.968592
3,TEST_01+1순,36629.492046,656239.058778,169086.279218,1832.800358,12112.184487,6055.495841,21915.174283,1007.503102,1461.731116,32076.516548
4,TEST_01+2순,37043.118973,656706.187873,169157.955693,1807.629747,12773.803937,6252.754205,19345.095480,959.428487,1409.168055,34744.486630
...,...,...,...,...,...,...,...,...,...,...,...
70,TEST_23+2순,34582.583981,671002.007409,169157.955693,1807.629747,10694.315681,3937.315445,17368.937590,941.120165,1477.520127,31902.733077
71,TEST_23+3순,44066.810727,642479.834673,168874.425219,1811.650465,10672.809641,4003.321632,18424.741820,978.935053,1339.270492,31205.987733
72,TEST_24+1순,44592.667173,547162.231031,169086.279218,1676.924577,14005.301385,8168.957702,28805.907222,1094.649010,546.058152,38868.923376
73,TEST_24+2순,35324.506936,552230.575566,169157.955693,1807.629747,12671.638372,7308.773331,28200.826495,1238.405026,562.146624,37003.599185
